In [ ]:
from langchain.tools import tool
from langchain_exa import ExaSearchRetriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from langchain.agents import create_agent
import chromadb
import requests
from langgraph.checkpoint.memory import InMemorySaver
from langchain.chat_models import init_chat_model
import time
from threading import Lock
from collections import deque
load_dotenv()

EXA_API_KEY = os.getenv("EXA_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID") 
CHROME_API_KEY = os.getenv("CHROME_API_KEY")
CHROMA_API_KEY = os.getenv("CHROMA_API_KEY")


model = ChatOpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    model="openai/gpt-5-nano",
    temperature=0.1
)


In [42]:
exa_retriever = ExaSearchRetriever(api_key=EXA_API_KEY, k=3, highlights=True)
client = chromadb.CloudClient(
  api_key=CHROMA_API_KEY,
  tenant='361f16d2-3a10-4479-854c-519de88ae973',
  database='job_search_db'
)
embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={'device': 'cpu'}
        )
vector_store = Chroma(
    client=client,
    collection_name="skills_jobs",
    embedding_function=embeddings
)
chroma_retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":5})


In [ ]:
# Add at the top of your cell with the tools


# Rate limiter class
class ExaRateLimiter:
    def __init__(self, max_requests: int = 5, time_window: float = 1.0):
        self.max_requests = max_requests
        self.time_window = time_window
        self.request_times = deque()
        self.lock = Lock()
    
    def acquire(self):
        """Wait if necessary to stay within rate limit"""
        with self.lock:
            now = time.time()
            # Remove requests older than the time window
            while self.request_times and self.request_times[0] < now - self.time_window:
                self.request_times.popleft()
            
            # If we're at the limit, wait until we can make another request
            if len(self.request_times) >= self.max_requests:
                sleep_time = self.request_times[0] + self.time_window - now
                if sleep_time > 0:
                    time.sleep(sleep_time)
                    now = time.time()
                    while self.request_times and self.request_times[0] < now - self.time_window:
                        self.request_times.popleft()
            
            # Record this request time
            self.request_times.append(now)

# Create rate limiter instance
exa_rate_limiter = ExaRateLimiter(max_requests=5, time_window=1.0)

# Then modify your exa_search tool:
@tool
def exa_search(query: Annotated[str, "The query to execute to find key summary information."]):
    """Use Exa Search to find key summary information"""
    exa_rate_limiter.acquire()  # Rate limit before API call
    results = exa_retriever.invoke(query)
    return results

In [ ]:


@tool
def google_search(query: Annotated[str, "The boolean search query to execute to find key summary information."]):
    """Given a google boolean search query, return the top 10 results

    
    """
    params = {
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "q": query,
        # Optional: restrict to last month -> d = days, w = weeks, m = months, y = years
        "num": 10,             # 1..10 per page
        "safe": "off",
        "lr": "lang_en",       # optional language
    }
    resp = requests.get("https://www.googleapis.com/customsearch/v1", params=params, timeout=20)
    resp.raise_for_status()
    data = resp.json()  
    
    results = [
    {
        "title": item.get("title"),
        "link": item.get("link"),
        "snippet": item.get("snippet"),
        "displayLink": item.get("displayLink"),
    }
        for item in data.get("items", [])
    ]
    
    return results

@tool
def match_jobs(query:Annotated[str,"The skills and job title the user wants to search for"]):
    """retrieve relevant jobs to help the user query"""
    results = chroma_retriever.invoke(query)
    return results




In [44]:

research_agent = create_agent(
    model,
    tools=[exa_search],
    system_prompt=(
        "You are an expert company research agent specializing in gathering comprehensive, accurate information about companies. "
        "Your role is to conduct thorough research and provide users with detailed insights about organizations.\n\n"
        
        "CORE RESPONSIBILITIES:\n"
        "• Analyze user queries to understand what company information is needed\n"
        "• Search for relevant, up-to-date information about companies\n"
        "• Synthesize information from multiple sources into coherent summaries\n"
        "• Provide actionable insights about company culture, values, and operations\n\n"
        
        "AVAILABLE TOOLS:\n"
        "1. exa_search: Use Exa Search to find key summary information, company news, updates, and relevant details\n\n"
        
        "RESEARCH STRATEGY:\n"
        "• Formulate precise search queries that capture the essence of what information is needed\n"
        "• Search for company background, recent news, culture, values, mission, and operations\n"
        "• Look for information about company size, industry, products/services, and reputation\n"
        "• Focus on factual, verifiable information rather than speculation\n\n"
        
        "OUTPUT FORMAT:\n"
        "Provide structured information including:\n"
        "• Company overview and background\n"
        "• Key facts (industry, size, location, etc.)\n"
        "• Recent news and developments\n"
        "• Company culture and values (if available)\n"
        "• Relevant insights for the user's query\n"
        "• Source citations for transparency\n\n"
        
        "BEST PRACTICES:\n"
        "• Verify information credibility and recency\n"
        "• Synthesize information from multiple sources when available\n"
        "• Focus on information that directly addresses the user's query\n"
        "• Present information clearly and concisely\n"
        "• Distinguish between facts and opinions when citing sources"
    ),
    name="reseracher"
)

tailor_agent = create_agent(
    model,
    tools=[match_jobs],
    system_prompt=(
        "You are an expert resume tailoring agent specializing in customizing resumes to match specific job descriptions. "
        "Your goal is to help users optimize their resumes by highlighting relevant skills and experiences that align with target positions.\n\n"
        
        "CORE RESPONSIBILITIES:\n"
        "• Analyze job descriptions to identify key requirements, skills, and qualifications\n"
        "• Compare user's resume against job requirements\n"
        "• Provide specific recommendations for resume customization\n"
        "• Suggest how to reframe experiences to better match job needs\n\n"
        
        "AVAILABLE TOOLS:\n"
        "1. match_jobs: Retrieve relevant job postings from the database based on skills and job title to understand requirements\n\n"
        
        "RESUME TAILORING STRATEGY:\n"
        "• Identify keywords and phrases from the job description that should appear in the resume\n"
        "• Match user's existing skills and experiences to job requirements\n"
        "• Suggest how to emphasize transferable skills and relevant accomplishments\n"
        "• Recommend adding or rephrasing bullet points to align with job needs\n"
        "• Ensure resume sections (summary, experience, skills) are optimized for the specific role\n\n"
        
        "OUTPUT FORMAT:\n"
        "Provide structured recommendations including:\n"
        "• Key requirements extracted from the job description\n"
        "• Match analysis: How the user's resume aligns with job requirements\n"
        "• Specific tailoring suggestions:\n"
        "  - Keywords to add or emphasize\n"
        "  - Skills to highlight\n"
        "  - Experiences to reframe or expand\n"
        "  - Sections to add or modify\n"
        "• Before/after examples for key sections (if applicable)\n\n"
        
        "BEST PRACTICES:\n"
        "• Be specific and actionable in your recommendations\n"
        "• Maintain authenticity - don't suggest fabricating experience\n"
        "• Focus on relevant skills and transferable experiences\n"
        "• Prioritize the most important requirements from the job description\n"
        "• Explain why each suggestion improves the resume's match to the job"
    ),
    name="tailor"
)

job_matching_agent = create_agent(
    model,
    tools=[match_jobs, google_search],
    system_prompt=(
        "You are an expert job matching agent specializing in finding the most relevant job opportunities for users. "
        "Your primary goal is to analyze user queries and provide comprehensive, targeted job search results.\n\n"
        
        "CORE RESPONSIBILITIES:\n"
        "• Analyze user skills, experience, preferences, and career goals\n"
        "• Search for jobs that match the user's profile and requirements\n"
        "• Provide detailed job recommendations with clear reasoning\n"
        "• Present results in a structured, easy-to-understand format\n\n"
        
        "AVAILABLE TOOLS:\n"
        "1. match_jobs: Search your internal database for existing job postings that match the user's query\n"
        "2. google_search: Perform targeted web searches for additional job opportunities\n\n"
        
        "SEARCH STRATEGY:\n"
        "For Google searches, use advanced boolean operators to refine results:\n"
        "• Use quotes for exact phrases: \"software engineer\"\n"
        "• Use OR for alternatives: (\"python\" OR \"java\") AND \"developer\"\n"
        "• Use site: operator to target specific job boards\n"
        "• Combine location, skills, and job titles effectively\n\n"
        
        "TARGET JOB BOARDS:\n"
        "Focus searches on these high-quality job platforms:\n"
        "• boards.greenhouse.io\n"
        "• ashbyhq.com\n"
        "• jobs.lever.co\n"
        "• jobs.smartrecruiters.com\n"
        "• wd1.myworkdayjobs.com\n"
        "• jobs.bamboohr.com\n"
        "• jobs.jobvite.com\n"
        "• careers.icims.com\n"
        "• apply.jazz.co\n"
        "• careers.workable.com\n\n"
        
        "OUTPUT FORMAT:\n"
        "For each job recommendation, provide:\n"
        "• Job title and company name\n"
        "• Location (remote/onsite/hybrid)\n"
        "• Key requirements and qualifications\n"
        "• Why this job matches the user's profile\n"
        "• Application link or source\n\n"
        
        "BEST PRACTICES:\n"
        "• Always explain your reasoning for job matches\n"
        "• Prioritize recent job postings when possible\n"
        "• Consider both exact matches and transferable skills\n"
        "• Provide actionable next steps for applications\n"
        "• Be specific about requirements and qualifications\n\n"
        
        "Remember: Your goal is to help users find jobs that align with their career aspirations and qualifications, not just any available positions."
    ),
    name="job_matcher"
)

@tool
def research(request: str) -> str:
    """Research a company
    Use this when the user wants to research a company
    
    Input: Natural Language Query about a company
    """
    result = research_agent.invoke({"messages": [{"role": "user", "content": request}]})
    return result["messages"][-1].text

@tool
def tailor(request: str) -> str:
    """Tailor a resume
    Use this when the user wants to tailor their resume to a specific job description
    
    Input: Natural Language Query about a job description
    """
    result = tailor_agent.invoke({"messages": [{"role": "user", "content": request}]})
    return result["messages"][-1].text

@tool
def match_jobs(request: str) -> str:
    """Match jobs to the user's resume
    Use this when the user wants to find jobs that match their resume
    
    Input: Natural Language Query about a job title and skills
    """
    result = job_matching_agent.invoke({"messages": [{"role": "user", "content": request}]})
    return result["messages"][-1].text







supervisor_agent = create_agent(
    model,
    tools=[research, tailor, match_jobs],
    system_prompt=(
        "You are an intelligent supervisor agent responsible for coordinating a team of specialized agents to help users with their job search and career needs. "
        "Your role is to analyze user queries and delegate tasks to the most appropriate agent.\n\n"
        
        "CORE RESPONSIBILITIES:\n"
        "• Analyze user queries to understand their intent and needs\n"
        "• Determine which agent(s) should handle the request\n"
        "• Delegate tasks to the appropriate agent(s) in the correct sequence\n"
        "• Monitor task completion and coordinate multi-step workflows\n"
        "• Synthesize results from multiple agents when needed\n\n"
        
        "AVAILABLE AGENTS:\n"
        "1. research_agent: Specializes in researching companies, gathering information about organizations, company culture, values, news, and background details\n"
        "2. tailor_agent: Specializes in customizing resumes to match specific job descriptions, providing resume optimization recommendations\n"
        "3. job_matching_agent: Specializes in finding relevant job opportunities based on user skills, experience, and preferences\n\n"
        
        "DELEGATION STRATEGY:\n"
        "• Analyze the user query to identify the primary intent:\n"
        "  - Company research requests → research_agent\n"
        "  - Resume tailoring/optimization requests → tailor_agent\n"
        "  - Job search/finding opportunities → job_matching_agent\n"
        "  - Multi-step requests → delegate to agents in logical sequence\n"
        "• For complex queries that require multiple agents:\n"
        "  - Delegate to one agent at a time (not in parallel)\n"
        "  - Wait for each agent's response before proceeding\n"
        "  - Use results from previous agents to inform next steps\n"
        "  - Synthesize final results for the user\n\n"
        
        "WORKFLOW EXAMPLES:\n"
        "• \"Find software engineer jobs\" → job_matching_agent\n"
        "• \"Research company X\" → research_agent\n"
        "• \"Tailor my resume for this job\" → tailor_agent\n"
        "• \"Find jobs at Google and tailor my resume\" → job_matching_agent → tailor_agent\n"
        "• \"Research Microsoft and find relevant jobs\" → research_agent → job_matching_agent\n\n"
        
        "BEST PRACTICES:\n"
        "• Always delegate to one agent at a time - do not call agents in parallel\n"
        "• Clearly communicate the task and context to each agent\n"
        "• Review agent responses before proceeding to next steps\n"
        "• Provide clear, synthesized final responses to users\n"
        "• If a query is unclear, ask for clarification before delegating\n"
        "• Ensure each agent has all necessary context to complete their task\n\n"
        
        "Remember: Your goal is to orchestrate the team effectively to provide users with comprehensive, accurate, and actionable assistance for their job search needs."
    ),
)

In [45]:
user_request = "Research Microsoft and then find relevant software engineer jobs there"
    


for step in supervisor_agent.stream(
        {"messages": [{"role": "user", "content": user_request}]}
    ):
        for update in step.values():
            for message in update.get("messages", []):
                message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  research (call_HFkfuDwJ5vTwGEOXUn73jU2R)
 Call ID: call_HFkfuDwJ5vTwGEOXUn73jU2R
  Args:
    request: Research Microsoft: provide a concise company overview including history, core products and services, culture and values, leadership, recent major news, and business highlights that affect hiring (e.g., growth areas, locations, teams, and engineering focus).


ValueError: Request failed with status code 429: {"error":"You've exceeded your Exa rate limit of 5 requests per second. If you want this increased, please email hello@exa.ai :)"}